In [1]:
import cv2
import os
import mediapipe as mp
import timeit
import pandas as pd
import numpy as np

import json
import matplotlib.pyplot as plt


import jk_module
import sys
sys.path.append('D:/bagel/action_assessment/jk_OKS.py')
import jk_OKS
import jk_draw

In [16]:
frame2 = cv2.imread('img.png')
if frame2 is None:
    print(f"Failed to load image from {image_dir}")
else:
    print("Image loaded successfully")

Image loaded successfully


In [22]:

import cv2
import os
import mediapipe as mp
import timeit
import pandas as pd
import numpy as np

import json
import matplotlib.pyplot as plt


import jk_module
'''import sys
sys.path.append('D:/bagel/action_project2/jk_OKS.py')'''
import jk_OKS
import jk_draw

import time
'''
import jk_vector
import jk_angle
import jk_DTW


import ast
import csv
from dtaidistance import dtw'''



## Trainer 경로
raw_root_dir = 'G:/공유 드라이브/R&D/DEX-001 (DEXULIN)/AI 자료/0.연구자료/action_assessment_data/' 

trainer_video_name = 'trainer_keypoints.mp4'
trainer_video_dir = os.path.join(raw_root_dir,trainer_video_name)

cap0 = cv2.VideoCapture(trainer_video_dir)
width0 = int(cap0.get(cv2.CAP_PROP_FRAME_WIDTH))
height0 = int(cap0.get(cv2.CAP_PROP_FRAME_HEIGHT))

## User 경로
for trial in range(2, 4):
    user_video_name = f'user/user_{trial}.mp4'
    user_video_dir = os.path.join(raw_root_dir,user_video_name)

    cap1 = cv2.VideoCapture(user_video_dir)
    width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
    height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Trainer 비디오 파일 열기
    cap0 = cv2.VideoCapture(trainer_video_dir)
    if cap0.isOpened():
        print("ok")
    else:
        print("failed")

    # 카메라 장치 열기

    if cap1.isOpened():
        print("ok")
    else:
        print("failed")

    # 두 비디오의 크기를 동일하게 설정 (더 작은 크기로 조정)
    frame_width = min(width0, width1)
    frame_height = min(height0, height1)


    # frame 크기 변경하여 시각화 및 저장(입력은 입력 이미지 사용)
    # 입력 (1080,1920)
    new_width = 1240
    new_height = 720
    
    # 경로 설정
    data_dir = os.path.join(raw_root_dir, 'data')

    # data 폴더가 없으면 생성
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    # VideoWriter 객체 생성을 위한 설정
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    trainer_output_video_name = f'trainer_output_{trial}.mp4'
    trainer_video_output_dir = os.path.join(data_dir, trainer_output_video_name)
    out = cv2.VideoWriter(trainer_video_output_dir, fourcc, 12.0, (new_width, new_height))

    '''# VideoWriter 객체 생성을 위한 설정
    user_output_video_name = f'user_{trial}.mp4'
    user_video_output_dir = os.path.join(raw_trainer_root_dir, user_output_video_name)
    user_out = cv2.VideoWriter(user_video_output_dir, fourcc, 3.0, (width1 , height1 ))
    '''
    # 미디어 파이프 모델 선택
    mp_drawing = mp.solutions.drawing_utils 
    mp_pose = mp.solutions.pose

    # score board
    image_name = 'image.png'
    image_dir = os.path.join(raw_root_dir,image_name)
    frame2 = cv2.imread('img.png')

    # trainer 데이터 로드
    csv_results_name = 'csv/trainer_results.csv'
    csv_results_path =  os.path.join(raw_root_dir,csv_results_name)
    
    csv_world_results_name = 'csv/trainer_world_results.csv'
    csv_world_results_path =  os.path.join(raw_root_dir,csv_world_results_name)
    
    trainer_results_df = pd.read_csv(csv_results_path)
    trainer_world_results_df = pd.read_csv(csv_world_results_path)

    # user 데이터
    user_base_results_df = trainer_results_df.copy()
    user_world_results_df = trainer_world_results_df.copy()

    row = 0

    # main code
    with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.9) as pose:
        '''previous_frame0 = None
        previous_frame1 = None'''
        
        while True:
            ret0, frame0 = cap0.read()
            if not ret0:
                print("Failed to read from cap0")
                break
            ret1, frame1 = cap1.read()
            if not ret1:
                print("Failed to read from cap1")
                break
            # 시작 시점 측정
            start_t = timeit.default_timer()
            
            #############################################################
            ## Trainer 이미지 처리 (image0)
            image0 = frame0
            image1 = frame1
            
            
            ############################################################
            ## User 이미지 처리 (image1)
            # Flip frame1 horizontally
            #frame1_flipped = cv2.flip(frame1, 1)
            frame1_flipped_resized = cv2.resize(frame1, (822, 586))
            
            try:
                # Now use frame1_flipped for further processing
                image1, results1 = jk_module.mediapipe_detection(frame1_flipped_resized, pose)
                mp_drawing.draw_landmarks(image1, results1.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(0,66,230), thickness=6, circle_radius=4), 
                                        mp_drawing.DrawingSpec(color=(0,0,255), thickness=6, circle_radius=4)) 
                landmarks = results1.pose_landmarks.landmark
                world_landmarks = results1.pose_world_landmarks.landmark
                
                #csv_results_path = f'user_results{trial}.csv'
                #csv_world_results_path =  f'user_world_results1{trial}.csv'
                #jk_module.landmarks_save(csv_results_path, results1, row)
                #jk_module.world_landmarks_save(csv_world_results_path, results1, row)

                # Continue with the rest of your code, ensuring to use the flipped image where necessary
            except:
                pass
            ############################################################
            ## 데이터 로드
            if row >= len(trainer_world_results_df):
                break
                
            now_trainer_world = trainer_world_results_df.iloc[row].tolist()
            now_trainer_base = trainer_results_df.iloc[row].tolist()

            now_user_world = [coord for w_landmark in world_landmarks for coord in (w_landmark.x, w_landmark.y, w_landmark.z, w_landmark.visibility)]
            #now_user_base = [coord for landmark in landmarks for coord in (landmark.x, landmark.y, landmark.z, landmark.visibility)]

            now_user_base = []
            for landmark in landmarks:
                now_user_base.extend([landmark.x , landmark.y, landmark.z, landmark.visibility])
            
            user_world_results_df.loc[row] = now_user_world
            user_base_results_df.loc[row] = now_user_base
            
            
            
            ###평가
            ############################################################
            ##2. score board 이미지 처리 (image2)
            image2 = cv2.resize(frame2, (frame_width, frame_height))

            """입력"""
            columns = [23,25,11,13,15,27, 24,26,12,14,16,28]
            #OKS_list = jk_OKS.OKS_score(now_trainer_world, now_user_world, columns)
            OKS_list_base = jk_OKS.OKS_2d_score(now_trainer_base, now_user_base, columns)
            #jk_OKS.OkS_text(image2, OKS_list)

            jk_OKS.OkS_text_base3(image2, OKS_list_base)


            ############################################################
            ##3. visual 이미지 처리 (image3)
            image3 = np.ones((frame_height, frame_width, 3), dtype=np.uint8) * 255
            
            image3 = jk_draw.draw_person2(image3, now_trainer_base, frame_width, frame_height)
            image3 = jk_draw.draw_person2(image3, now_user_base, frame_width, frame_height, color_keypoint=(0,66,230), color_connection=(0,0,255))
                    

            ############################################################
            ##4. concat 두 프레임을 좌우 결합
            # Ensure all images are resized to the same height and width before concatenation
            image0_resized = cv2.resize(image0, (frame_width, frame_height))
            image1_resized = cv2.resize(image1, (frame_width, frame_height))
            image2_resized = cv2.resize(image2, (frame_width, frame_height))
            image3_resized = cv2.resize(image3, (frame_width, frame_height))

            # Concatenate the resized images
            combined_frame = np.hstack((image0_resized, image1_resized, image2_resized, image3_resized))
            

            # 프레임의 크기 조정
            resized_frame = cv2.resize(combined_frame, (new_width, new_height))
            ############################################################
            ##5.시간 측정
            # 종료 시점 측정
            end_t = timeit.default_timer()
            FPS = 1./(end_t - start_t )

            #cv2.putText(resized_frame,f'FPS: {FPS:.2f}', (300,50), cv2.FONT_ITALIC, 1, (255,0,0), 2)
            
            # 조정된 프레임을 파일로 저장
            out.write(resized_frame)
            #user_out.write(frame1_flipped)
            cv2.imshow('test', resized_frame)
            
            #time.sleep(0.1)
            
            row = row + 1

            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

            
        cap0.release()
        cap1.release()
        out.release()  
        #user_out.release()
        cv2.destroyAllWindows()

ok
ok
Failed to read from cap0
ok
ok
Failed to read from cap0


In [36]:

import cv2
import os
import mediapipe as mp
import timeit
import pandas as pd
import numpy as np

import json
import matplotlib.pyplot as plt


import jk_module
'''import sys
sys.path.append('D:/bagel/action_project2/jk_OKS.py')'''
import jk_OKS
import jk_draw

import time
'''
import jk_vector
import jk_angle
import jk_DTW


import ast
import csv
from dtaidistance import dtw'''



## Trainer 경로
raw_root_dir = 'G:/공유 드라이브/R&D/DEX-001 (DEXULIN)/AI 자료/0.연구자료/action_assessment_data/' 

trainer_video_name = 'trainer_keypoints.mp4'
trainer_video_dir = os.path.join(raw_root_dir,trainer_video_name)

cap0 = cv2.VideoCapture(trainer_video_dir)
width0 = int(cap0.get(cv2.CAP_PROP_FRAME_WIDTH))
height0 = int(cap0.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Trainer 비디오 파일 열기
cap0 = cv2.VideoCapture(trainer_video_dir)
if cap0.isOpened():
    print("ok")
else:
    print("failed")

## User 경로
cap1 = cv2.VideoCapture(0)
width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 카메라 장치 열기
if cap1.isOpened():
    print("ok")
else:
    print("failed")

# 두 비디오의 크기를 동일하게 설정 (더 작은 크기로 조정)
frame_width = min(width0, width1)
frame_height = min(height0, height1)


# frame 크기 변경하여 시각화 및 저장(입력은 입력 이미지 사용)
# 입력 (1080,1920)
new_width = 1240
new_height = 720

# 경로 설정
data_dir = os.path.join(raw_root_dir, 'data')

# data 폴더가 없으면 생성
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

trial = 0

# VideoWriter 객체 생성을 위한 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(f'trainer_output_all_{trial}.mp4', fourcc, 12.0, (new_width, new_height))

# VideoWriter 객체 생성을 위한 설정
user_out = cv2.VideoWriter(f'user_{trial}.mp4', fourcc, 3.0, (width0 , height0 ))


# 미디어 파이프 모델 선택
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose

# score board
image_name = 'image.png'
image_dir = os.path.join(raw_root_dir,image_name)
frame2 = cv2.imread('img.png')

    # trainer 데이터 로드
csv_results_name = 'csv/trainer_results.csv'
csv_results_path =  os.path.join(raw_root_dir,csv_results_name)

csv_world_results_name = 'csv/trainer_world_results.csv'
csv_world_results_path =  os.path.join(raw_root_dir,csv_world_results_name)

trainer_results_df = pd.read_csv(csv_results_path)
trainer_world_results_df = pd.read_csv(csv_world_results_path)

# user 데이터
user_base_results_df = trainer_results_df.copy()
user_world_results_df = trainer_world_results_df.copy()

row = 0

# main code
with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.9) as pose:
    '''previous_frame0 = None
    previous_frame1 = None'''
    
    while True:
        ret0, frame0 = cap0.read()
        if not ret0:
            print("Failed to read from cap0")
            break
        ret1, frame1 = cap1.read()
        if not ret1:
            print("Failed to read from cap1")
            break
        # 시작 시점 측정
        start_t = timeit.default_timer()
            
        #############################################################
        ## Trainer 이미지 처리 (image0)
        image0 = frame0
        image1 = frame1
        
        
        ############################################################
        ## User 이미지 처리 (image1)
        # Flip frame1 horizontally
        frame1_resized = cv2.resize(frame1, (frame0.shape[1], frame0.shape[0]))
        frame1_flipped_resized = cv2.flip(frame1_resized, 1)
        
        try:
            # Now use frame1_flipped for further processing
            image1, results1 = jk_module.mediapipe_detection(frame1_flipped_resized, pose)
            mp_drawing.draw_landmarks(image1, results1.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(0,66,230), thickness=6, circle_radius=4), 
                                    mp_drawing.DrawingSpec(color=(0,0,255), thickness=6, circle_radius=4)) 
            landmarks = results1.pose_landmarks.landmark
            world_landmarks = results1.pose_world_landmarks.landmark
            
            #csv_results_path = f'user_results{trial}.csv'
            #csv_world_results_path =  f'user_world_results1{trial}.csv'
            #jk_module.landmarks_save(csv_results_path, results1, row)
            #jk_module.world_landmarks_save(csv_world_results_path, results1, row)

            # Continue with the rest of your code, ensuring to use the flipped image where necessary
        except:
            pass
        ############################################################
        ## 데이터 로드
        if row >= len(trainer_world_results_df):
            break
            
        now_trainer_world = trainer_world_results_df.iloc[row].tolist()
        now_trainer_base = trainer_results_df.iloc[row].tolist()

        now_user_world = [coord for w_landmark in world_landmarks for coord in (w_landmark.x, w_landmark.y, w_landmark.z, w_landmark.visibility)]
        #now_user_base = [coord for landmark in landmarks for coord in (landmark.x, landmark.y, landmark.z, landmark.visibility)]

        now_user_base = []
        for landmark in landmarks:
            now_user_base.extend([landmark.x , landmark.y, landmark.z, landmark.visibility])
        
        user_world_results_df.loc[row] = now_user_world
        user_base_results_df.loc[row] = now_user_base
        
            
            
        ###평가
        ############################################################
        ##2. score board 이미지 처리 (image2)
        image2 = cv2.resize(frame2, (frame_width, frame_height))

        """입력"""
        columns = [23,25,11,13,15,27, 24,26,12,14,16,28]
        #OKS_list = jk_OKS.OKS_score(now_trainer_world, now_user_world, columns)
        OKS_list_base = jk_OKS.OKS_2d_score(now_trainer_base, now_user_base, columns)
        #jk_OKS.OkS_text(image2, OKS_list)

        jk_OKS.OkS_text_base3(image2, OKS_list_base)


        ############################################################
        ##3. visual 이미지 처리 (image3)
        image3 = np.ones((frame_height, frame_width, 3), dtype=np.uint8) * 255
        
        image3 = jk_draw.draw_person2(image3, now_trainer_base, frame_width, frame_height)
        image3 = jk_draw.draw_person2(image3, now_user_base, frame_width, frame_height, color_keypoint=(0,66,230), color_connection=(0,0,255))
                

        ############################################################
        ##4. concat 두 프레임을 좌우 결합
        # Ensure all images are resized to the same height and width before concatenation
        image0_resized = cv2.resize(image0, (frame_width, frame_height))
        image1_resized = cv2.resize(image1, (frame_width, frame_height))
        image2_resized = cv2.resize(image2, (frame_width, frame_height))
        image3_resized = cv2.resize(image3, (frame_width, frame_height))

        # Concatenate the resized images
        combined_frame = np.hstack((image0_resized, image1_resized, image2_resized, image3_resized))
        

        # 프레임의 크기 조정
        resized_frame = cv2.resize(combined_frame, (new_width, new_height))
        ############################################################
        ##5.시간 측정
        # 종료 시점 측정
        end_t = timeit.default_timer()
        FPS = 1./(end_t - start_t )

        # 조정된 프레임을 파일로 저장
        out.write(resized_frame)
        user_out.write(frame1_resized)
        
        cv2.imshow('test', resized_frame)
        

        row = row + 1

        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

        
    cap0.release()
    cap1.release()
    out.release()  
    user_out.release()
    cv2.destroyAllWindows()

ok
ok
